In [1]:
# Librerías:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import numpy as np

In [2]:
df3 = pd.read_excel('./Datos/Originales/df_dealroom_modif.xlsx')
lista_urls = list(i for i in df3['profile_url'])
lista_urls

['https://app.dealroom.co/companies/hostalia',
 'https://app.dealroom.co/companies/irizar_e_mobility',
 'https://app.dealroom.co/companies/haizea_wind',
 'https://app.dealroom.co/companies/solarpack',
 'https://app.dealroom.co/companies/lookiero',
 'https://app.dealroom.co/companies/instant_sport',
 'https://app.dealroom.co/companies/educaedu',
 'https://app.dealroom.co/companies/ajl_ophthalmic',
 'https://app.dealroom.co/companies/naivan',
 'https://app.dealroom.co/companies/novargi',
 'https://app.dealroom.co/companies/blue_agro',
 'https://app.dealroom.co/companies/ibil',
 'https://app.dealroom.co/companies/bas_projects',
 'https://app.dealroom.co/companies/glefaran',
 'https://app.dealroom.co/companies/cafdigitalservices',
 'https://app.dealroom.co/companies/lufe',
 'https://app.dealroom.co/companies/iline_microsystems',
 'https://app.dealroom.co/companies/vivebiotech',
 'https://app.dealroom.co/companies/electrorecycler',
 'https://app.dealroom.co/companies/owasys',
 'https://app.

Selenium y BeautifulSoup

In [20]:
names = []
launch_date = []
employees = []
ownership = []
location = []
ranking = []
    
driver = webdriver.Firefox('/home/bdata2/Descargas/geckodriver-v0.32.2-linux64')
    
for url in lista_urls[0:19]:
        
    #driver.maximize_window()
    time.sleep(8)
    driver.get(url)
    time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
        
    names.append(soup.find_all("h1", {"class", "name"})[0].text)
    if not soup.find("div", {"class", "description"}).find_all('time'):
        launch_date.append(np.NAN)
    else:
        #Se quitan los meses de Launch date y se dejan solo los años
        launch_date.append(str(soup.find("div", {"class", "description"}).find_all('time')[0].text).replace('Jan','').replace('Feb','').replace('Mar','').replace('Apr','').replace('May','').replace('Jun','').replace('Jul','').replace('Aug','').replace('Sep','').replace('Oct','').replace('Nov','').replace('Dec',''))
    if not soup.find_all("div", {"class", "field employees"}):
        employees.append(np.NAN)
    else:
        employees.append(str(soup.find("div", {"class", "field employees"}).find_all('a')[0].text).replace('people',''))
    if not soup.find_all("a", {"class", "field-value-link"}):
        ownership.append(np.NAN)
    else:
        ownership.append(soup.find_all("a", {"class", "field-value-link"})[0].text)
    if not soup.find_all("span", {"class", "highlight"}):
        location.append(np.NAN)
    else:
        location.append(str(soup.find_all("span", {"class", "highlight"})[0].text).replace('Basque Country','').replace('View more',''))
        #HQ : oficina central
    if not soup.find_all("div", {"class", "ranking-section-container"}):
        ranking.append(np.NAN)
    else:
        ranking.append(str(soup.find_all("div", {"class", "ranking-section-container"})[0].text).replace('Dealroom.co signal',''))
    
dic = {'Names': names, 'Launch date':launch_date,'Employees': employees,'Ownership':ownership,'Location':location,'Ranking':ranking}

In [21]:
dt = pd.DataFrame.from_dict(dic, orient='index').transpose()
dt

,Names,Launch date,Employees,Ownership,Location,Ranking
0,Hostalia,2002,11-50,bootstrapped,Bilbao (HQ),10
1,Irizar E-Mobility,2014,51-200,accelerator,Aduna (HQ),15
2,Haizea Wind,2016,51-200,NaN,Zierbena (HQ),16
3,Solarpack,2005,201-500,public,Getxo (HQ & founding location),11
4,Lookiero,2016,201-500,venture capital,Bilbao (HQ),47
5,Wavegarden,2005,51-200,NaN,San Sebastián (HQ),22
6,Educaedu,2001,51-200,private equity,Bilbao (HQ),NaN
7,AJL Ophthalmic,1992,51-200,accelerator,Vitoria (HQ & founding location),12
8,Naivan,NaN,11-50,NaN,Vitoria (HQ),11
9,NOVARGI,2004,11-50,NaN,Vitoria (HQ),12


In [23]:
dt.isna().sum()

Names          0
Launch date    1
Employees      2
Ownership      8
Location       0
Ranking        3
dtype: int64

In [24]:
group_employees = dt.groupby(["Employees"])["Names"].count().reset_index().rename(columns={'Names':'Num Names'})
group_ownership = dt.groupby(["Ownership"])["Names"].count().reset_index().rename(columns={'Names':'Num Names'})
group_location = dt.groupby(["Location"])["Names"].count().reset_index().rename(columns={'Names':'Num Names'})
print(group_employees)
print(group_ownership)
print(group_location)

  Employees  Num Names
0    11-50           5
1     2-10           2
2  201-500           2
3   51-200           8
         Ownership  Num Names
0      accelerator          6
1     bootstrapped          1
2   private equity          1
3           public          1
4  venture capital          2
                             Location  Num Names
0                         Aduna  (HQ)          1
1                      Azpeitia  (HQ)          1
2                     Barakaldo  (HQ)          1
3                        Bilbao  (HQ)          4
4                      Galdakao  (HQ)          1
5     Getxo  (HQ & founding location)          1
6                 San Sebastián  (HQ)          5
7   Vitoria  (HQ & founding location)          1
8                       Vitoria  (HQ)          2
9                         Zalla  (HQ)          1
10                     Zierbena  (HQ)          1


Función para buscar información de la empresa escribiendo su nombre.

In [303]:
def find_by_name(name):
    if name in list(dt['Names']):
        index_num = [index for index, dato in enumerate(list(dt['Names'])) if dato == name]
        info = dt.loc[index_num]
    return(info)

# Ejemplo con Hostalia
find_by_name('Hostalia')

,Names,Launch date,Employees,Ownership,Location
0,Hostalia,2002,11-50,bootstrapped,Bilbao (HQ)
